In [12]:
import json
import os

def read_jsonl(path:str):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f.readlines() if  line]


In [13]:
# from google.colab import drive
# drive.mount('/content/drive')

In [14]:
path = os.path.join("test.jsonl")
test_data = read_jsonl(path)

In [15]:
import warnings
warnings.filterwarnings("ignore")
import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [16]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer=AutoTokenizer.from_pretrained("./Qwen2-0.5B",padding_side = "left")
model=AutoModelForCausalLM.from_pretrained("./Qwen2-0.5B").to(device)

# tokenizer=AutoTokenizer.from_pretrained("../Record/Qwen2-0.5B-Instruct")
# model=AutoModelForCausalLM.from_pretrained("../Record/Qwen2-0.5B-Instruct").to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
from tqdm import tqdm
import json
import torch

# 假设test_data是一个包含问题的列表
# 假设tokenizer和model已经定义好并且在设备上

predictions = []

# 这里设置batch_size
batch_size = 32
num_batches = (len(test_data) + batch_size - 1) // batch_size

for i in tqdm(range(num_batches)):
    batch_data = test_data[i * batch_size: (i + 1) * batch_size]
    batch_msgs = []
    for question in batch_data:
        eight_shot = """Question: Bert, Ernie, and Peggy collect stamps. Bert has four times as many stamps as Ernie, but Ernie has three times as many stamps as Peggy. If Peggy currently has 75 stamps in her collection, how many stamps does she need to add to her collection to have a collection as large as Bert's collection?\nAnswer: Ernie has three times as many stamps as Peggy, or 3*75=<<3*75=225>>225 stamps.\nBert has 4 times as many stamps as Ernie, or 4*225=<<4*225=900>>900 stamps.\nThus, Peggy would need 900-75=<<900-75=825>>825 more stamps to have a collection as large as Bert's.\n#### 825\n\n"""
        msg = [
            {"role": "system", "content": "you are a helpful assistant"},
            {"role": "user", "content": f"{eight_shot}Question:{question['question']}\n\nAnswer:"},
        ]
        text = tokenizer.apply_chat_template(
            msg,
            tokenize=False,
            add_generation_prompt=True,
            pad_token_id=tokenizer.pad_token_id,
        )
        batch_msgs.append(text)
    model_inputs = tokenizer(batch_msgs, padding=True, return_tensors="pt", truncation=True).to(device)
    generated_ids = model.generate(
        model_inputs.input_ids,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=256,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    predictions.extend(responses)
    del model_inputs
    torch.cuda.empty_cache()

100%|██████████| 42/42 [07:10<00:00, 10.26s/it]


In [20]:
import json


json_predictions = [{"question":question['question'],"answer": prediction} for prediction,question in zip(predictions,test_data)]

with open('Base_without_CoT.json', 'w') as f:
    json.dump(json_predictions, f, indent=4)